In [1]:
import warnings
warnings.filterwarnings('ignore')
import sys
sys.path.append("../subjects/")
import pickle
import matplotlib.pyplot as plt
import itertools
from sklearn.cluster import KMeans
import tensorflow as tf
import time
import random, math
import tensorflow_probability as tfb
import pandas as pd
import numpy as np
from sklearn.metrics import f1_score, accuracy_score
from sklearn.metrics.pairwise import  euclidean_distances
import glob
import re
from Utils_Functions import generate_dataset, eod
from sklearn.model_selection import train_test_split
dataset ='Adult'
for dataset in ['Adult']:#,'Compas','Bank','Heart','Law','Student']:
    if dataset == 'Adult':
        sens_index = 7
        priv_group = 1
        unpriv_group = 0
        data_file_name = 'adult_org-Copy1.csv'
    if dataset == 'Compas':
        sens_index = 1
        priv_group = 1
        unpriv_group = 0
        data_file_name = 'compas-Copy1'
    if dataset == 'Bank':
        sens_index = 0
        priv_group = 5
        unpriv_group = 3
        data_file_name = 'bank'
    if dataset == 'Heart':
        sens_index = 0
        priv_group = 1
        unpriv_group = 0 
        data_file_name = 'heart_processed_1'
    if dataset == 'Law':
        sens_index = 1
        priv_group = 1
        unpriv_group = 0
        data_file_name = 'law.csv'

    if dataset == 'Student':
        sens_index = 0
        priv_group = 1
        unpriv_group = 0  
        data_file_name = 'students-processed_2'
    df = pd.read_csv('../subjects/datasets/'+data_file_name)
    df = df.drop_duplicates()
    
    
    
    
    X = df.to_numpy()[:,:-1]
    Y1 = df.to_numpy()[:,-1]
    
    acc_test_rnd = []
    cluster_acc=[]
    for num_cluster in range(1,1000,5):
        acc_test =[]
        for i in range(1):
            X1, X_test = train_test_split(X)
    
            
            KMean = KMeans(n_clusters=num_cluster)
            KMean.fit(X1)
    
            if dataset=='Adult':
                cont_cols=['hr']
            elif dataset=='Law':
                cont_cols=['UGPA']
    
            rnd_df = pd.DataFrame(columns=df.columns)
    
            for col in rnd_df.columns:
                if col in cont_cols:
                    rnd_df[col] = np.random.uniform(df[col].min(),df[col].max(),X_test.shape[0])
                else:
                    rnd_df[col] = np.random.randint(df[col].min(),df[col].max(),X_test.shape[0])
            ave_dist =[] 
            for i in range(KMean.n_clusters):
                mean_dist = euclidean_distances(X1[np.where(KMean.labels_==[i])],[KMean.cluster_centers_[i]]).mean()
                std_dist = euclidean_distances(X1[np.where(KMean.labels_==[i])],[KMean.cluster_centers_[i]]).std()
                if dataset == 'Heart':
                    ave_dist.append(mean_dist+ (3 * std_dist))
                elif dataset == 'Compas':
                    ave_dist.append(mean_dist+ (1 * std_dist))
                elif dataset == 'Student':
                    ave_dist.append(mean_dist+ (1 * std_dist))
                else:
                    ave_dist.append(mean_dist)
            final_res=[]
            for i in range(X_test.shape[0]):
                if euclidean_distances([X_test[i]],KMean.cluster_centers_).min() <= max(ave_dist):
                    final_res.append(1)
                else :
                    final_res.append(0)
            final_res = np.array(final_res)
            acc_test.append(np.where(np.array(final_res)==1)[0].shape[0]/final_res.shape[0])
            # print(dataset, acc_test[-1])   
    
            X_test_rnd = rnd_df.to_numpy()[:,:-1]
            final_res_rnd=[]
            for i in range(X_test_rnd.shape[0]):
                if euclidean_distances([X_test_rnd[i]],KMean.cluster_centers_).min() <= max(ave_dist):
                    final_res_rnd.append(1)
                else :
                    final_res_rnd.append(0)
            final_res_rnd = np.array(final_res_rnd)
            acc_test_rnd.append(np.where(np.array(final_res_rnd)==1)[0].shape[0]/final_res_rnd.shape[0])
            # print(dataset+' RND', acc_test_rnd[-1])
        # print('Acc test',np.mean(acc_test) )
        # print('Acc rnd test',np.mean(acc_test_rnd) )
        cluster_acc.append(np.mean(acc_test))
        print(num_cluster,cluster_acc[-1])

1 0.7998729351969505
6 0.929987293519695
11 0.9443456162642948
16 0.9381194409148665
21 0.9325285895806862
26 0.9426937738246506
31 0.9135959339263025
36 0.9353240152477764
41 0.9279542566709021
46 0.921728081321474
51 0.955146124523507
56 0.9486658195679797
61 0.9407878017789072
66 0.9526048284625159
71 0.9411689961880559
76 0.9364675984752223
81 0.956289707750953
86 0.9475222363405337
91 0.9458703939008895
96 0.941423125794155
101 0.9609911054637865
106 0.949428208386277
111 0.9411689961880559
116 0.955146124523507
121 0.9576874205844981
126 0.9383735705209657
131 0.9466327827191868
136 0.9501905972045743
141 0.9453621346886912
146 0.9613722998729352
151 0.9548919949174078
156 0.9692503176620076
161 0.9490470139771283
166 0.9625158831003812
171 0.9593392630241423
176 0.9495552731893265
181 0.9565438373570521
186 0.9616264294790343
191 0.9623888182973317
196 0.9719186785260483
201 0.9611181702668361
206 0.9481575603557815
211 0.9702668360864041
216 0.9632782719186785
221 0.97242693773

In [4]:
np.save('./cluster_acc.npy',cluster_acc)